# Earnings Conference Call Transcript

## Imports

In [1]:
# System
import os
import requests

# LLM Models
from langchain_openai import ChatOpenAI

# Templates
from langchain_core.prompts import ChatPromptTemplate

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

from bs4 import BeautifulSoup
from docx import Document
import markdown

# Gradio
import gradio as gr

# Alpha Vantage
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper

## Load Environment Variables

In [2]:
from dotenv import load_dotenv
load_dotenv()

os.environ['USER_AGENT'] = 'JimYin88'

In [3]:
advantage_api_key = os.getenv('ALPHAVANTAGE_API_KEY')
alpha_vantage = AlphaVantageAPIWrapper()

In [4]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_completion_tokens=4096,
                        temperature=0.3)

In [5]:
def markdown_to_docx(markdown_text):
    # Convert Markdown to HTML
    html = markdown.markdown(markdown_text)

    # Create a new Word Document
    html_content = ''

    # Add the HTML content
    for line in html.splitlines():
        html_content += line + '\n'

    doc = Document()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Convert HTML elements to Word document elements
    for element in soup:
        if element.name == 'h1':
            doc.add_heading(element.get_text(), level=1)
        elif element.name == 'h2':
            doc.add_heading(element.get_text(), level=2)
        elif element.name == 'h3':
            doc.add_heading(element.get_text(), level=3)
        elif element.name == 'p':
            doc.add_paragraph(element.get_text())
        elif element.name == 'ul':
            for li in element.find_all('li'):
                doc.add_paragraph(li.get_text(), style='ListBullet')
        elif element.name == 'ol':
            for li in element.find_all('li'):
                doc.add_paragraph(li.get_text(), style='ListNumber')
        # Add more HTML elements as needed

    return doc

In [ ]:
def get_company_ticker(company_name='NVDA'):

    prompt = ChatPromptTemplate.from_template(
        """
        Provide the company stock ticker based on the following company name: {raw_text}.
        If the stock ticker is already provided, Just return the stock ticker, and nothing else.
        """
    )

    chat_chain = prompt | chat_model | StrOutputParser()
    response = chat_chain.invoke({"raw_text": company_name})

    return response

In [19]:
def earnings_conference_call(company_name='NVDA',
                             fiscal_year=2025,
                             quarter='Q1',
                             output_format='Transcript'):

    tick = get_company_ticker(company_name).upper()
    yr = str(int(fiscal_year))
    filename = f'{tick.lower()}_{yr}{quarter}_{output_format.lower()}.docx'

    url = (f'https://www.alphavantage.co/query?function=EARNINGS_CALL_TRANSCRIPT&symbol={tick}&quarter={yr}'
           f'{quarter}&apikey={advantage_api_key}')

    r = requests.get(url)

    if r.status_code == 200:
        data = r.json()

        if output_format == 'Summary':
            result = ''
            result += f"Ticker: {data['symbol']}\n"
            result += f"Fiscal Quarter: {data['quarter']}\n\n"
            for section in data['transcript']:
                result += f"Speaker: {section['speaker']}\nTitle: {section['title']}\n\n{section['content']}\n\n"

            prompt = ChatPromptTemplate.from_template(
                """
                Provide a summary of the following company earnings conference call: {raw_text}.
                Have the title of conference call followed by a list of speakers and their titles after their names. 
                Then have sections and key points in each section in bullet points. 
                Add a conclusion section at the end. Have the output in markdown format. 
                Do not include the text 'markdown' or the triple (''') in the output. 
                Do not have bullet points with no text.
                """
            )

            chat_chain = prompt | chat_model | StrOutputParser()
            response = chat_chain.invoke({"raw_text": result})
            word_docx = markdown_to_docx(response)
            word_docx.save(filename)

            print(f"File {filename} saved:")
            return filename

        else:
            document = Document()
            document.add_heading('Company Earnings Conference Call', level=0)
            document.add_heading(f"Ticker: {data['symbol']}\nQuarter: FY{data['quarter']}\n", level=1)
            for section in data['transcript']:
                p = document.add_paragraph("")
                p.add_run(f"Speaker: {section['speaker']}\nTitle: {section['title']}").bold = True
                document.add_paragraph(f"{section['content']}")

            document.save(filename)

        print(f"File {filename} saved:")
        return filename

    else:
        print(f"Request failed with status code: {r.status_code}")
        return None

In [20]:
interface = gr.Interface(fn=earnings_conference_call,
                         inputs=[gr.Text(label="Company Name or Stock Ticker",
                                         value='',
                                         placeholder="Type in the Company Name or Stock Ticker"),
                                 gr.Slider(minimum=2006, maximum=2026, value=2025, step=1, label='Fiscal Year'),
                                 gr.Dropdown(choices=['Q1', 'Q2', 'Q3', 'Q4'],
                                             value='Q1',
                                             multiselect=False,
                                             allow_custom_value=False,
                                             label='Quarter'),
                                 gr.Dropdown(choices=['Summary', 'Transcript'],
                                             value='Transcript',
                                             multiselect=False,
                                             allow_custom_value=False,
                                             label='Output Type')],
                         outputs=[gr.File(label="Output")],
                         flagging_mode="never",
                         theme=gr.themes.Base(),
                         title='Company Earnings Conference Call Retrieval',
                         description="""
                                     This app can retrieve either the raw transcript or a summary of company earnings 
                                     conference call. Enter the company's stock ticker, the fiscal year, the quarter 
                                     (Q1, Q2, Q3, or Q4), and the type of output (either Summary or Transcript). 
                                     Click on 'Submit' to download the file.                         
                                     """)

In [21]:
if __name__ == '__main__':
    interface.launch(share=False)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


C:\Users\JimYi\anaconda3\envs\langchain\Lib\site-packages\docx\styles\styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


File msft_2025Q1_summary.docx saved:


In [22]:
interface.close()

Closing server running on port: 7861
